In [2]:
import pandas as pd

# Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [52]:
raiting = pd.read_csv('ratings.csv')


In [51]:
time_user = raiting.groupby('userId').agg(['min', 'max'])['timestamp']  #сортируем таблицу по пользователям и выводим min и max значения времени посещения
time_user['lifetime'] = (time_user['max'] - time_user['min'])/3600/24   #добавляем столбец со временем жизни (дни) пользователя       

rating_user = raiting.groupby('userId').count()             #кол-во оценок каждого из пользователей 
rating_user = rating_user[(rating_user['rating'] > 100)]    #перезадание таблицы пользователей с кол-вом оценок > 100
rating_user.head(20)

rating_time = rating_user.join(time_user)                  #слияние таблиц ("левое" слияние - сохраняются id пользователей в rating_user)
time_average = m['lifetime'].mean()                                #вычисление среднего по столбцу lifetime

print(time_average)

463.89476214829165


# Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
1. Таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
2. Аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [53]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [54]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [55]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [56]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [65]:
revenue = (rzd.merge(auto, how = 'outer', on = 'client_id').      #слияние таблиц rzd и auto
    merge(air, how = 'outer', on = 'client_id').fillna(0))        #слияние таблиц rzd+auto и air. fillna(0) - замена пустых значений на 0

revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [70]:
revenue_address = revenue.merge(client_base, how = 'outer', on = 'client_id')

revenue_address

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


# Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
1. У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
3. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Ответ:
    Создать сводную таблицу визитов и покупок и добавить данные геолокации по user_id. Если в данных по геолокации есть привязка ко времени, то можно их связать с фактом покупки. В результате обработки координат пользователей можно оценить потребности рынка в той или иной продукции.